Na podstwie ostatnich 5 przewidzieć kolejną jedna -- wejsscie to deltta t i zmiana kursu



Dane zrobic w fromie 5 danych i wynik , potemm je poshuflowac
wyrzucic zmiany dluzsze niz dzien
mozna zsporbowac dane gdzie nic sie nie zmienilo tez wyrzucic 

# TO DO:

1. zmienic close na cours change w prepare_day DONE
2. zamineic df na numpy array DONE
3. przetestowac wyjatek w prepare_day
4. podzielic miesiace na grupy 6,1,3,1 DONE
4. przetestowac inne modele, w liniowym sprobowac zrobic binaryny output

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path='../data/KGHM_clear.csv'
df=pd.read_pickle(path)

In [ ]:
LEARN_SAMPLE_SIZE=5

In [ ]:
df

In [ ]:
df.head()

In [ ]:
# df=df[:100000]

# Grupowanie po dniach

In [ ]:
gb=df.groupby(df.day)

df_group_by_day_list=[gb.get_group(x).copy() for x in gb.groups]

for i in df_group_by_day_list:
    i.reset_index

In [ ]:
df_group_by_day_list[0]

# Dla każdego dnia możemy przeprowadzić osobne operacje

In [ ]:
def prepare_day(df_group_by_day_list,learn_sample_size=5,predict_next=True):
    
    if predict_next:
        predict_n_steps_future=1
    else:
        predict_n_steps_future=0
        raise ValueError("Predicting long then next doesnt work yet") 
    for i,df_group in enumerate(df_group_by_day_list):
        #resetowanie indeksow
        
        
        #dodawanie wczesniejszych wartosci pomiarow
        for j in range(1,learn_sample_size):
            df_group_by_day_list[i]['CoursChange-'+str(j)]=df_group_by_day_list[i].loc[:,('CoursChange')].shift(j)
            df_group_by_day_list[i]['TimeDelta_-'+str(j)]=df_group_by_day_list[i].TimeDelta.shift(j)
        
        #tworzenie wynikow
        df_group_by_day_list[i].loc[:,('y')]=df_group_by_day_list[i].loc[:,('CoursChange')].shift(-predict_n_steps_future)
        df_group_by_day_list[i].drop(df_group_by_day_list[i].head(learn_sample_size).index,inplace=True)
        df_group_by_day_list[i].drop(df_group_by_day_list[i].tail(predict_n_steps_future).index,inplace=True)
        df_group_by_day_list[i].reset_index(inplace=True,drop=True)
        #         df_group_by_day_list[i].drop(df_group_by_day_list[i][df_group_by_day_list[i].TimeDelta>4000].index)
#     Next step -> delate outstandig values


In [ ]:
prepare_day(df_group_by_day_list,learn_sample_size=LEARN_SAMPLE_SIZE)

In [ ]:
df_group_by_day_list[0]

In [ ]:
df_merged=pd.concat(df_group_by_day_list, axis=0, ignore_index=True)

In [ ]:
df_merged.month.min()

In [ ]:
maximum=df_merged.month.max()
test=[]
train=[]
i=df_merged.month.min()
while i<(df_merged.month.max()+1):
    for j in range(i,i+6):
        train.append(j)
    i+=6
    i+=1
    for j in range(i,i+3):
        test.append(j)
    i+=3
    i+=1
    
    

In [ ]:
gb=df_mergedrged.groupby(df.month)

df_group_by_month_list=[gb.get_group(x).copy() for x in gb.groups]

for i in df_group_by_day_list:
    i.reset_index

In [ ]:
test_df=[]
train_df=[]
for i in range(len(df_group_by_day_list)):
    x=df_group_by_day_list[i].month[0]
    if x in test:
        test_df.append(df_group_by_day_list[i])
    if x in train:
        train_df.append(df_group_by_day_list[i])
    

In [ ]:
df_test=pd.concat(test_df, axis=0, ignore_index=True)
df_train=pd.concat(train_df, axis=0, ignore_index=True)

# Prawodpodobnie udało się podzielić na miesiące zgodnie ze wskazaniami 
# (6 train, 1 przerwy, 3 test, 1 przerwy)

In [ ]:
x=df_test.month.unique()
x.sort()
x

In [ ]:
x=df_train.month.unique()
x.sort()
x

# Wygląda, że działa!

In [ ]:
# df_merged[df_merged[df_merged.TimeDelta>8000].index[0]-2:df_merged[df_merged.TimeDelta>8000].index[0]+2]

In [ ]:
fig, ax = plt.subplots()
df_merged.TimeDelta.hist(ax=ax,bins=100, bottom=0.1)

# series.hist( bins=100, bottom=0.1)
ax.set_yscale('log')
ax.set_title('TimeDelta after removing days breaks')
plt.show()

# Prediction

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, model_selection, svm
import numpy as np

In [ ]:
def df_to_array(df,shuffle=True):
    '''
    Takes dateframe and returns numpy arrays X,y
    '''
    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)
    X = np.array(df[[x for x in df.columns if 'Time' in x or 'Cours' in x]])
    y =  np.array(df.y)
    return X, y

In [ ]:
X_test,y_test=df_to_array(df_test)
X_train,y_train=df_to_array(df_train)

In [ ]:
y_train.shape

In [ ]:
y_test.shape

## Add normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_test)
rescaledX_test= scaler.transform(X_test)
rescaledX_train=scaler.transform(X_train)
# summarize transformed data
np.set_printoptions(precision=3)

In [ ]:
X_train=rescaledX_train
X_test=rescaledX_test

In [ ]:
# Training
clf = LinearRegression()
clf.fit(X_train,y_train)
# Testing
confidence = clf.score(X_test, y_test)
print("confidence: ", confidence)

In [ ]:
forecast_prediction = clf.predict(X_forecast)
print(forecast_prediction)

In [ ]:
a=X_test[:-5]
print(a)
print(y_test[:-5])

In [ ]:
clf.predict(a)